## 그룹 연산

#### 데이터 집계
- groupby 메소드로 평균값 구하기

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:/STUDY/Pandas/gapminder.tsv', sep='\t')

In [3]:
avg_life_exp_by_year = df.groupby('year')['lifeExp'].mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


- groupby 메소드의 분할/반영/결합 과정 확인하기

In [6]:
# 분할 작업
years = df['year'].unique()   # = df.year.unique()
print(years) 

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [11]:
# 반영 작업
year_means = []
for y in years:
    m = df.loc[df.year == y, : ].lifeExp.mean()
    year_means.append((y,m))
print(year_means)

[(1952, 49.05761971830987), (1957, 51.507401126760534), (1962, 53.60924901408449), (1967, 55.67828957746479), (1972, 57.647386478873244), (1977, 59.57015746478873), (1982, 61.53319718309858), (1987, 63.21261267605636), (1992, 64.16033802816901), (1997, 65.01467605633805), (2002, 65.69492253521126), (2007, 67.00742253521126)]


In [10]:
# 결합 작업
df2 = pd.DataFrame({
    'year' : [y for y, m in year_means],
    '' : [m for y, m in year_means]
})
df2

,year,
0,1952,49.057620
1,1957,51.507401
2,1962,53.609249
3,1967,55.678290
4,1972,57.647386
5,1977,59.570157
6,1982,61.533197
7,1987,63.212613
8,1992,64.160338
9,1997,65.014676


- 사용사 함수 적용 agg(): apply()와 유사

In [16]:
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
        return sum / n

In [18]:
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    0.202824
1957    0.213606
1962    0.225331
1967    0.239577
1972    0.254141
1977    0.270690
1982    0.280662
1987    0.287479
1992    0.293479
1997    0.294106
2002    0.296683
2007    0.308648
Name: lifeExp, dtype: float64


- 2개의 인자 사용

In [19]:
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

In [21]:
global_mean = df.lifeExp.mean()
print(global_mean)

59.47443936619714


In [22]:
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean)
print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


In [23]:
import numpy as np

In [24]:
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
gdf

,count_nonzero,mean,std
year,,,
1952,142,49.057620,12.225956
1957,142,51.507401,12.231286
1962,142,53.609249,12.097245
1967,142,55.678290,11.718858
1972,142,57.647386,11.381953
1977,142,59.570157,11.227229
1982,142,61.533197,10.770618
1987,142,63.212613,10.556285
1992,142,64.160338,11.227380


In [25]:
gdf_dict = df.groupby('year').agg({'lifeExp':'mean', 'pop' : 'median', 'gdpPercap':'median'})
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371108


#### 데이터 변환
- 표준점수 계산하기

In [26]:
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [27]:
trans_z = df.groupby('year').lifeExp.transform(my_zscore)
print(trans_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [ ]:
# agg = year 기준으로 묶어서 값을 반환
# transform = 통계컬럼에서 각각의 요소들로 반환

- 누락값을 평균값으로 처리하기

In [28]:
import seaborn as sns
import numpy as np

In [30]:
np.random.seed(1234)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10

,total_bill,tip,sex,smoker,day,time,size
187,30.46,2.00,Male,Yes,Sun,Dinner,5
179,34.63,3.55,Male,Yes,Sun,Dinner,2
31,18.35,2.50,Male,No,Sat,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
71,17.07,3.00,Female,No,Sat,Dinner,3
6,8.77,2.00,Male,No,Sun,Dinner,2
95,40.17,4.73,Male,Yes,Fri,Dinner,4
131,20.27,2.83,Female,No,Thur,Lunch,2
157,25.00,3.75,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4


-  누락값을 평균값으로 처리하기

In [31]:
tips_10.loc[np.random.permutation(tips_10.index)[:4],'total_bill']=np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
187,30.46,2.00,Male,Yes,Sun,Dinner,5
179,34.63,3.55,Male,Yes,Sun,Dinner,2
31,NaN,2.50,Male,No,Sat,Dinner,4
52,NaN,5.20,Female,No,Sun,Dinner,4
71,NaN,3.00,Female,No,Sat,Dinner,3
6,8.77,2.00,Male,No,Sun,Dinner,2
95,40.17,4.73,Male,Yes,Fri,Dinner,4
131,20.27,2.83,Female,No,Thur,Lunch,2
157,25.00,3.75,Female,No,Sun,Dinner,4
5,NaN,4.71,Male,No,Sun,Dinner,4


In [32]:
# 흡연자와 비흡연자 평균 따로 계산
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

tb_grp_mean = tips_10.groupby('smoker').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = tb_grp_mean
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
187,30.46,2.00,Male,Yes,Sun,Dinner,5,30.460000
179,34.63,3.55,Male,Yes,Sun,Dinner,2,34.630000
31,NaN,2.50,Male,No,Sat,Dinner,4,18.013333
52,NaN,5.20,Female,No,Sun,Dinner,4,18.013333
71,NaN,3.00,Female,No,Sat,Dinner,3,18.013333
6,8.77,2.00,Male,No,Sun,Dinner,2,8.770000
95,40.17,4.73,Male,Yes,Fri,Dinner,4,40.170000
131,20.27,2.83,Female,No,Thur,Lunch,2,20.270000
157,25.00,3.75,Female,No,Sun,Dinner,4,25.000000
5,NaN,4.71,Male,No,Sun,Dinner,4,18.013333


- apply / agg / transform

In [33]:
# 숫자로만 구성된 데이터프레임
def plus(value):
    return value + 2

In [36]:
df = pd.DataFrame({'a':[10,20,30], 'b':[20,30,40]})
df

,a,b
0,10,20
1,20,30
2,30,40


In [42]:
print(df.apply(plus))
print(df.agg(plus))
print(df.transform(plus))

    a   b
0  12  22
1  22  32
2  32  42
    a   b
0  12  22
1  22  32
2  32  42
    a   b
0  12  22
1  22  32
2  32  42


In [46]:
print(df.a.apply(plus))
print(df.a.agg(plus))
print(df.a.transform(plus))

0    12
1    22
2    32
Name: a, dtype: int64
0    12
1    22
2    32
Name: a, dtype: int64
0    12
1    22
2    32
Name: a, dtype: int64


- 집계함수
> apply 데이터프레임만 가능 / agg 데이터프레임/시리즈 가능 /  trangform 둘다 불가

In [47]:
# 적용시킬 사용자 함수 내에서 집계 함수 사용
def mean(value):
    return value.min()

In [49]:
df.apply(mean)
df.agg(mean)

a    10
b    20
dtype: int64

In [50]:
df.a.agg(mean)

10

In [51]:
df.a.apply(mean)

AttributeError: 'int' object has no attribute 'min'

In [52]:
# ValueError
df.transform(mean)
df.a.transform(mean)

ValueError: Function did not transform

In [55]:
# 숫자 데이터 열과 문자 데이터 열이 함께 있는 데이터프레임
df2 = pd.DataFrame({
    'a' : [10,20,30,40],
    'b' : [20,30,40,50],
    'c' : ['서울','대전','부산','제주'],
    'd' : ['Female','Male','Male','Female']
})
df2

,a,b,c,d
0,10,20,서울,Female
1,20,30,대전,Male
2,30,40,부산,Male
3,40,50,제주,Female


In [56]:
df2.apply(mean)

a        10
b        20
c        대전
d    Female
dtype: object

In [62]:
# 그룹화한 데이터
df2_grp = df2.groupby('c')
df2_grp.transform(plus)

C:\Users\admin\AppData\Local\Temp\ipykernel_8708\2375783567.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  df2_grp.transform(plus)


,a,b
0,12,22
1,22,32
2,32,42
3,42,52


In [63]:
df2_grp.apply(mean)

,a,b,c,d
c,,,,
대전,20,30,대전,Male
부산,30,40,부산,Male
서울,10,20,서울,Female
제주,40,50,제주,Female


In [64]:
df2_grp.agg(mean)

,a,b,d
c,,,
대전,20,30,Male
부산,30,40,Male
서울,10,20,Female
제주,40,50,Female


In [65]:
df2_grp.transform(mean)

,a,b,d
0,10,20,Female
1,20,30,Male
2,30,40,Male
3,40,50,Female
